In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/kaggle/input/gufhtugu-publications-dataset-challenge/GP Orders - 5.csv")
mydata = data
mydata.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# FOR FUZZY MATCH
import fuzzywuzzy
from fuzzywuzzy import process, fuzz

# FOR TRANSLATE URDU-ENGLISH
from textblob import TextBlob

In [ ]:
mydata = mydata.rename({"Order Number": "OrdNumber", 
                        "Order Status": "OrdStatus", 
                        "Book Name" : "BookName", 
                        "Order Date & Time": "OrdDate", 
                        "City": "city", 
                        "Payment Method": "PayMethod", 
                        "Total items": "TotItems", 
                        "Total weight (grams)": "weight"}, 
                       axis = 1 )
mydata.city =mydata.city.astype('str')
mydata.city = mydata.city.str.lower()
mydata.city = mydata.city.str.strip()
short_names = mydata.city.str.len()<=4

In [ ]:
short_to_full = {
    "fsd": "faisalabad",
    "rwp": "rawalpindi",
    "lhr": "lahore",
    "isb": "islamabad",
    "ryk": "rahim yar khan",    
    "khi": "karachi",
    "hyd": "hydrabad",
    "aj&k": "azad jammu & kashmir",
    "lahr": "lahore",
    "lah": "lahore",
    "d i khan": "dera ismail khan",
    "g g khan": "dera ghazi khan",
    "dgk": "dera ghazi khan",
    "vehari": "vihari",
    "mailsi, district vehari": "vihari"
}
mydata.city = mydata.city.replace(short_to_full)

In [ ]:
alpha_num_name = mydata.city.str.contains("[a-zA-Z0-9]")
beore_urduNames = mydata[~alpha_num_name].city.unique()
convert_to_eng = {}
for x in beore_urduNames:
    try:
        tr = TextBlob(x).translate().string
        convert_to_eng[x] = tr
    except:
        pass
mydata.city = mydata.city.replace(convert_to_eng)

In [ ]:
alpha_num_name = mydata.city.str.contains("[a-zA-Z0-9]")
after_urduNames = mydata[~alpha_num_name].city.unique()
after_urduNames.shape, after_urduNames

In [ ]:
remaing_translate_to_eng = {
    "خانیوال": "khanewal",
    "ہیلاں": "heelan",
    "اوکاڑہ": "okara",
    "پھالیہ": "phalia",
    "پیرمحل": "pir mahal",
    "لیہ": "layyah",
    "سنانواں": "sananwan",
    "جعفراباد": "jaffarabad",
    "کوہیٹا": "kohita",
    "روہیلانوالی": "rohilanwali",
    "ہرنائی": "harnai",
    "ہنگو": "hangu",
    "محلہ گوڑھا": "mohalla godha",
    "ٹمن": "timon",
    "ہرنولی": "harnoli",
    "سکہر": "sukkur",
    "جام پور": "jampur",
    "ميرپورخاص": "mirpurkhas",
    "لالیاں": "lalian",
    "بھکر": "bhakkar",
    "پلندری": "pallandri",
    "کنڈیارو": "kandiaro",
    "دھوک مصاحب": "dhok musahib",
    "صالح پٹ": "saleh Pat",
    "لودھراں": "lodhran",
    "پتوکی": "patoki",
    "چوک آعظم": "chowk azam"
}
mydata.city = mydata.city.replace(remaing_translate_to_eng)
alpha_num_name = mydata.city.str.contains("[a-zA-Z0-9]")
after_replace_urduNames = mydata[~alpha_num_name].city.unique()
after_replace_urduNames.shape

Use External Data for Cities Normalization

In [ ]:
externalData = pd.read_csv("../input/pakistan-cities/pk.csv")
externalData.city = externalData.city.str.lower()
externalData.city = externalData.city.str.strip()
externalData[externalData.city.str.contains('city')]

In [ ]:
externalData.city = externalData.city.replace('sialkot city', 'sialkot')
externalData.city = externalData.city.replace('jhang city', 'jhang')
externalData.city = externalData.city.replace('hyderabad city', 'hyderabad')
externalData.city = externalData.city.replace('attock city', 'attock')

In [ ]:
testExt = {}
mydata.new_city = None
mydata.lat = None
mydata.lng = None
mydata.ratio = None

for x, xRows in externalData.iterrows():
    for y, yRows in mydata.iterrows():
        if (xRows.city not in yRows.city):
            if xRows.city not in testExt:
                testExt[xRows.city] = {"count":1}
            else:
                testExt[xRows.city]["count"] = testExt[xRows.city]["count"] + 1
        else:
            getRatio = fuzz.ratio(yRows.city, xRows.city)
            mydata.at[y, 'ratio'] = getRatio
            mydata.at[y, 'new_city'] = xRows.city
            mydata.at[y, 'lat'] = xRows.lat
            mydata.at[y, 'lng'] = xRows.lng
mydata.info()
mydata

In [ ]:
print ("\nTotal Number of Rows: ", len(mydata))
print ("\n\n----------------- BEFORE PROCESSING --------------------\n")
print ("Unique Cities: ", len(mydata['city'].unique()))
print (f"Total Unique Cities: {mydata['city'].nunique()} " )
print ("\n\n------------------ AFTER PROCESSING --------------------\n")
print ("Unique Cities: ", len(mydata['new_city'].unique()))
print (f"Total Unique Cities: {mydata['new_city'].nunique()} " )
print ("\n\n---------- UNPROCCESSED/INVALID CITIES --------------\n")
print ("Number of Invalid / UnProccessed Cities: ", mydata['new_city'].isnull().sum())

# Visualiztion of Orders per Cities

In [ ]:
mydata_map = mydata
mydata_map.head()
mydata_map

# Total Number of Orders Per Cities

In [ ]:
cities_per_order = pd.DataFrame(columns=['city', 'orders', 'Longitude', 'Latitude'])
cities = list(mydata_map['new_city'].str.strip())
print ("Total Cities in dataFrame: ", len(cities))

In [ ]:
for city in cities:
    orderNos = 0
    iter_data = mydata_map[mydata_map['new_city'] == city]
    geolocations = iter_data.drop_duplicates(subset = ["lat","lng"])
    orderNos = len(iter_data)
    lat = geolocations['lat']
    lng = geolocations['lng']
    df = { "city": city, "orders": orderNos, "Longitude": lng, "Latitude": lat}
    cities_per_order = cities_per_order.append(pd.DataFrame(df))

In [ ]:
cities_per_order
print ("Total Cities in dataFrame: ", len(cities_per_order))
# Calculating Total Number of Orders
total_orders = len(mydata_map)
print ("Total Number of Orders: ", total_orders)

In [ ]:
#pip install folium
import folium as fo

In [ ]:
# with folium-map library
pakistan_map = fo.Map(location = [30.37,69.34],
                          zoom_start=5.5,
                          tiles='OpenStreetMap' )

for city, orders, lng, lat in zip(list(cities_per_order['city']), 
                                 list(cities_per_order['orders']), 
                                 list(cities_per_order['Longitude']), 
                                 list(cities_per_order['Latitude'])):
    fo.CircleMarker(location=[lat, lng], 
                        radius=(float(orders)/total_orders)*100, 
                        color="green", 
                        fill=True, opacity=0.8, 
                        fill_color="green").add_to(pakistan_map )
    
pakistan_map

# Number of Orders Status Per Cities

In [ ]:
cities_per_ordStatus = pd.DataFrame(columns=['city', 'ordComplete', 'ordCancel', 'ordReturn', 'Longitude', 'Latitude'])
mydata_map['OrdStatus'].unique()

In [ ]:
for city in cities:
    Ordcomplete = 0
    ordCancel = 0
    ordReturn = 0
    iter_data = mydata_map[mydata_map['new_city'] == city]
    Ordcomplete = len(iter_data[iter_data['OrdStatus'] == "Completed" ])
    ordCancel = len(iter_data[iter_data['OrdStatus'] == "Cancelled" ])
    ordReturn = len(iter_data[iter_data['OrdStatus'] == "Returned" ])
    geolocations = iter_data.drop_duplicates(subset = ["lat","lng"])
    lat = geolocations['lat']
    lng = geolocations['lng']
    df = { "city": city, "ordComplete": Ordcomplete, "ordCancel": ordCancel, "ordReturn": ordReturn, "Longitude": lng, "Latitude": lat}
    cities_per_ordStatus = cities_per_ordStatus.append(pd.DataFrame(df))

In [ ]:
cities_per_ordStatus

In [ ]:
orderStat_map = fo.Map(location = [30.37,69.34],
                          zoom_start=5.5,
                          tiles='OpenStreetMap' )

for city, ordComplete, ordCancel, ordReturn, lng, lat in zip(list(cities_per_ordStatus['city']), 
                                                             list(cities_per_ordStatus['ordComplete']), 
                                                             list(cities_per_ordStatus['ordCancel']), 
                                                             list(cities_per_ordStatus['ordReturn']), 
                                                             list(cities_per_ordStatus['Longitude']), 
                                                             list(cities_per_ordStatus['Latitude'])):
    fo.CircleMarker(location=[lat, lng], radius=(float(ordComplete)/total_orders)*100, 
                        color="green", fill=True, opacity=0.8, fill_color="green", 
                   tooltip='{}: {}'.format(city, ordComplete) ).add_to(orderStat_map )
    fo.CircleMarker(location=[lat, lng], radius=(float(ordCancel)/total_orders)*100, 
                        color="red", fill=True, opacity=0.8, fill_color="red", 
                   tooltip='{}: {}'.format(city, ordCancel) ).add_to(orderStat_map )
    fo.CircleMarker(location=[lat, lng], radius=(float(ordReturn)/total_orders)*100, 
                        color="purple", fill=True, opacity=0.8, fill_color="purple", 
                   tooltip='{}: {}'.format(city, ordReturn) ).add_to(orderStat_map )
    
orderStat_map

In [ ]:
print("Unique Payment Methods: ", mydata_map['PayMethod'].unique())
print("NULL Payment Methods: ", mydata_map['PayMethod'].isna().sum())
mydata_map['PayMethod'].value_counts()

In [ ]:
mydata_map['PayMethod'] = mydata_map['PayMethod'].replace('Cash on Delivery (COD)', 'Cash on delivery')
mydata_map.head()

In [ ]:
cities_paymethod = pd.DataFrame(columns=['city', 'COD', 'EasyPaisa', 'JazzCash', 'BankTransfer', 'Longitude', 'Latitude'])

mydata_pay = mydata_map[mydata_map['PayMethod'].notna()]
print("Unique Payment Methods: ", mydata_pay['PayMethod'].unique())
print("NULL Payment Methods: ", mydata_pay['PayMethod'].isna().sum())

total_paymethods = len(mydata_pay)
print("Total Pay Methods: ", total_paymethods)

mydata_pay['PayMethod'].value_counts()

In [ ]:
for city in cities:
    cod = 0
    epaisa = 0
    jcash = 0
    bTransfer = 0
    iter_data = mydata_map[mydata_map['new_city'] == city]
    cod = len(iter_data[iter_data['PayMethod'] == "Cash on delivery" ])
    epaisa = len(iter_data[iter_data['PayMethod'] == "EasyPaisa" ])
    jcash = len(iter_data[iter_data['PayMethod'] == "JazzCash" ])
    bTransfer = len(iter_data[iter_data['PayMethod'] == "BankTransfer" ])
    geolocations = iter_data.drop_duplicates(subset = ["lat","lng"])
    lat = geolocations['lat']
    lng = geolocations['lng']
    df = { "city": city, "COD": cod, "EasyPaisa": epaisa, "JazzCash": jcash, "BankTransfer": bTransfer, "Longitude": lng, "Latitude": lat}
    cities_paymethod = cities_paymethod.append(pd.DataFrame(df))

In [ ]:
cities_paymethod

In [ ]:
payMethod_map = fo.Map(location = [30.37,69.34],
                          zoom_start=5.5,
                          tiles='OpenStreetMap' )

for city, cod, epaisa, jcash, bTransfer, lng, lat in zip(list(cities_paymethod['city']), 
                                                         list(cities_paymethod['COD']), 
                                                         list(cities_paymethod['EasyPaisa']), 
                                                         list(cities_paymethod['JazzCash']), 
                                                         list(cities_paymethod['BankTransfer']), 
                                                         list(cities_paymethod['Longitude']), 
                                                         list(cities_paymethod['Latitude'])):
    fo.CircleMarker(location=[lat, lng], radius=(float(cod)/total_paymethods)*100, 
                        color="green", fill=True, opacity=0.8, fill_color="green", 
                       tooltip='{}: {}'.format(city, cod) ).add_to(payMethod_map )
    fo.CircleMarker(location=[lat, lng], radius=(float(epaisa)/total_paymethods)*100, 
                        color="red", fill=True, opacity=0.8, fill_color="red", 
                   tooltip='{}: {}'.format(city, epaisa) ).add_to(payMethod_map )
    fo.CircleMarker(location=[lat, lng], radius=(float(jcash)/total_paymethods)*100, 
                        color="purple", fill=True, opacity=0.8, fill_color="purple", 
                   tooltip='{}: {}'.format(city, jcash) ).add_to(payMethod_map )
    fo.CircleMarker(location=[lat, lng], radius=(float(bTransfer)/total_paymethods)*100, 
                        color="yellow", fill=True, opacity=0.8, fill_color="yellow", 
                   tooltip='{}: {}'.format(city, bTransfer) ).add_to(payMethod_map )
    
payMethod_map

Hope attempt is appriciated **with lots of love**, looking for your guidance **like, UPVOTES for motivation**
Previously Notebook comments and like/Upvores highly appriciated and motivated
Heartly Thanks with lots of **LOVE**